# 5.1 简单层的实现
- 反向传播利用计算图理解
- 基本原理是链式法则

In [ ]:
# 乘法层
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x*y
        return  out
    
    def  backward(self,dout):
        dx = dout*self.y
        dy = dout*self.x

        return dx,dy

# 测试
apple_price = 100
apple_num = 2
tax_consumer = 1.1
apple = MulLayer() #多层创建多个实例
tax = MulLayer()
# 正向传播
price = apple.forward(apple_price,apple_num)
price_tax = tax.forward(price,tax_consumer)
# 反向传播
dprice_tax = 1
dprice,dtax = tax.backward(dprice_tax)
dapple_price,dapple_num = apple.backward(dprice)
print(dapple_price,dapple_num,dtax)

In [ ]:
# 加法层的实现
class AddLayer:
    def __init__(self):
        pass

    def forward(self,x,y):
        out = x+y
        return out
    
    def backward(self,dout):
        dx  = dout*1
        dy = dout*1
        return dx, dy

# 测试
apple_price = 100
apple_num = 2
tax_con = 1.1
orange_price = 150
orange_num  =3
# forward
apple = MulLayer()
orange = MulLayer()
sum_fruit = AddLayer()
tax = MulLayer()
price_app = apple.forward(apple_price,apple_num)
price_ora = orange.forward(orange_price,orange_num)
price_fruit = sum_fruit.forward(price_app,price_ora)
price_tax = tax.forward(price_fruit,tax_con)
print(price_tax)
# backward
dprice_tax =1
dprice_fruit,dtax_con = tax.backward(dprice_tax)
print(dprice_fruit)
dprice_app,dprice_org = sum_fruit.backward(dprice_fruit)
dapple_price,dapple_num = apple.backward(dprice_app)
dorange_price,dorange_num = orange.backward(dprice_org)
print(dapple_price,dapple_num,dorange_price,dorange_num,dprice_fruit,dtax_con)

# 5.2 神经网络激活层的实现
- ReLU激活函数
- Sigmoid激活函数

In [ ]:
# ReLu激活函数
import numpy as np


class ReLu:
    '''激活函数的反向传播'''
    def __init__(self):
        self.mask = None
    
    def forward(self,x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        return dx

x = np.array([[1.0,-0.5],[-2.0,3.0]])
print(x)
f = ReLu()
out = f.forward(x)
dout  = np.ones_like(x)
dx = f.backward(dout)
print(out)
print(dx)

In [ ]:
# sigmoid激活函数

class Sigmoid:

    def __init__(self):
        self.out = None

    def forward(self,x):
        out = 1/(1+np.exp(-x))
        self.out = out

        return out
    
    def backward(self,dout):
        dx = dout*self.out*(1.0-self.out)

        return dx  

# 5.3 输入输出层的实现
- Affine层：线性变换
- Softmax层：多分类

In [ ]:
# Y=XW+B
# 对于XW的梯度，最后与X,W形状相同
# 对于B的梯度，按axis=0加总，因为B加到Y每一行上面，所以B对结果L的影响应该是Y每一行的影响之和

class Affine:
    def __init__(self,W,b):
        self.W=W
        self.b = b
        self.x =None
        self.dW = None
        self.db = None
    
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.W) +self.b

        return out
    
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db  = np.sum(dout,axis=0)

        return dx

In [ ]:
# Softmax层
def cross_entropy_error(y,t):
    if y.ndim == 1:
        t = t.reshape(1,t.size)
        y = y.reshape(1,y.size)
    
    batch_size = y.shape[0]
    return  -np.sum(t*np.log(y+1e-7))/batch_size


class SoftmaxWithLoss:

    def __init__(self):
        self.loss =None
        self.y = None
        self.t = None

    def softmax(x):
        a = np.max(x)
        exp_x = np.exp(x-a)
        exp_sum = np.sum(exp_x)
        y = exp_x/exp_sum
        return y

    def cross_entropy_error(y,t):

        if y.ndim == 1:
            t = t.reshape(1,t.size)
            y = y.reshape(1,y.size)
    
        batch_size = y.shape[0]
        return  -np.sum(t*np.log(y+1e-7))/batch_size
    
    def forward(self,x,t):
        self.t = t 
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        return self.loss
    
    def backward(self,dout=1):
        batch_size = self.t.shape[0] #批的大小
        if  self.t.size == self.y.size:
            dx = (self.y - self.t)/batch_size 
            #最后推导出来的公式/数据个数 将单个数据的误差传递给前面层
            # 如果有N份数据，那么计算的交叉熵对于x的导数，应该在一份数据的基础上1/N
        else:
            # 不是独热编码
            dx = self.y.copy()
            dx[np.arange(batch_size),self.t] -= 1
            # 花式索引，然后计算误差：概率-1
            dx = dx/batch_size
        return dx

# 5.4 误差反向传播法的实现
- 两层网络的实现

In [ ]:
from collections import OrderedDict


def numerical_gradient(f,x):
    '''梯度'''
    h =1e-4
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])   #多维迭代器
    # 不断按行迭代，计算每一行的梯度，直到完成
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = tmp_val +h
        fxh1 = f(x)
        x[idx] = tmp_val -h
        fxh2 = f(x)
        grad[idx] = (fxh1-fxh2)/(2*h)
        x[idx] = tmp_val
        it.iternext() 
    
    return grad


class TwoLayerNet:

    def __init__(self,input_size,hidden_size,output_size,weight_std = 0.01):
        # 初始化参数
        self.params = {}
        self.params['W1'] = weight_std*np.random.randn(input_size,hidden_size)
        self.params['b1'] =weight_std*np.random.randn(hidden_size)
        self.params['W2'] = weight_std*np.random.randn(hidden_size,output_size)
        self.params['b2'] = weight_std*np.random.randn(output_size)

        # 生成层
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'],self.params['b1'])
        self.layers['ReLu1'] = ReLu()
        self.layers['Affine2'] = Affine(self.params['W2'],self.params['b2'])
        # 输出层
        self.lastLayer = SoftmaxWithLoss()
    
    def predict(self,x):
        for layer in self.layers.values():  # 按有序字典循环层
            x = layer.forward(x)              # 每层都正向传播，得到得分
        
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def  accuracy(self,x,t):
        y= self.predict(x)
        y = np.argmax(y,axis=1)
        if  t.ndim != 1:     # 独热编码
            t = np.argmax(t,axis=1)
        
        accuracy = np.sum(y==t)/float(x.shape[0])
        # 如果是标签，那么最大的参数索引就是应该和t相等
        # 如果是独热编码，应该返回等于1的索引，与预测结果比较
        return accuracy
    
    def numerical_gradient(self,x,t):
        loss_W = lambda W: self.loss(x,t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W,self.params['W1'])
        grads['b1']  = numerical_gradient(loss_W,self.params['b1'])
        grads['W2']  = numerical_gradient(loss_W,self.params['W2'])
        grads['b2']  = numerical_gradient(loss_W,self.params['b2'])
        
        return  grads
        
    def gradient(self,x,t):
        # forward
        self.loss(x,t)
        # backward
        dout =1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()  # 列表反向
        for layer in layers:
            dout = layer.backward(dout)
    
        grads={ }
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads


In [ ]:
# 梯度确认
import sys,os
sys.path.append(os.pardir)
import numpy as np
from analysis.mnist import load_mnist

(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True,one_hot_label=True)
network = TwoLayerNet(784,50,10)

x_batch = x_train[:3]
t_batch = t_train[:3]
grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_back = network.gradient(x_batch,t_batch)
# 求绝对误差
for key in grad_numerical.keys():
    print(grad_numerical[key].shape)
    print(grad_back[key].shape)
    diff = np.average(np.abs(grad_numerical[key]-grad_back[key]))
    print(key+ " : " + str(diff))

In [ ]:
# 误差反向传播的学习
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate  = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size/batch_size,1)
(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True,one_hot_label=True)
network = TwoLayerNet(784,50,10)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch =  t_train[batch_mask]

    grad = network.gradient(x_batch,t_batch)
    # 更新
    for key in ("W1",'b1','W2','b2'):
        network.params[key] -= learning_rate*grad[key]
    
    loss = network.loss(x_batch,t_batch)
    train_loss_list.append(loss)
    if i%iter_per_epoch ==0:
        train_acc = network.accuracy(x_train,t_train)
        test_acc = network.accuracy(x_test,t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc,test_acc)

In [ ]:
# 可视化
import matplotlib.pylab as plt
x  = np.arange(len(train_acc_list))
y1  = np.array(train_acc_list)
y2 = np.array(test_acc_list)
plt.plot(x,y1,label='train_acc')
plt.plot(x,y2,label='test_acc',linestyle='--')
plt.show()